# VDEH Data Loading Pipeline

**Fokus:** Reine Datenextraktion aus OAI-PMH XML-Format

## 🎯 Ziel
- Parse VDEH XML-Datei (OAI-PMH Format)
- Extrahiere Grunddaten: Titel, Autoren, Jahr
- Validiere und bereinige Basisdaten
- Export als standardisierte Parquet-Datei

## 📚 Input/Output
- **Input**: `data/vdeh/raw/VDEH_mab_all.xml`
- **Output**: `data/vdeh/processed/01_loaded_data.parquet`

In [1]:
# 🛠️ SETUP: Initialize notebook environment
import sys
from pathlib import Path

# Add src to path (temporary until utils is imported)
project_root = Path.cwd()
while not (project_root / 'config.yaml').exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root / 'src'))

# Now use the utility function
from utils.notebook_utils import setup_notebook

project_root, config = setup_notebook()
print(f"✅ Project root: {project_root}")
print(f"✅ Project: {config.get('project.name')} v{config.get('project.version')}")

2025-11-10 19:52:12 - utils.notebook_utils - INFO - Searching for project root...


2025-11-10 19:52:12 - utils.notebook_utils - INFO - Project root found: /media/sz/Data/Bibo/analysis


2025-11-10 19:52:12 - utils.notebook_utils - INFO - Loading configuration...


2025-11-10 19:52:12 - config_loader - INFO - Configuration loaded from /media/sz/Data/Bibo/analysis/config.yaml


2025-11-10 19:52:12 - utils.notebook_utils - INFO - Configuration loaded successfully: Dual-Source Bibliothek Bestandsvergleich


✅ Project root: /media/sz/Data/Bibo/analysis
✅ Project: Dual-Source Bibliothek Bestandsvergleich v2.0.0


In [2]:
# 📦 IMPORTS
import pandas as pd
import warnings

# Configure warnings
if not config.get('debug.verbose_output', True):
    warnings.filterwarnings('ignore')

# Display VDEH data source configuration
vdeh_config = config.get('data_sources.vdeh')
print(f"📄 VDEH Datenquelle: {vdeh_config['description']}")
print(f"📂 Dateipfad: {config.project_root / vdeh_config['path']}")
print(f"🔤 Encoding: {vdeh_config['encoding']}")
print(f"📊 Geschätzte Records: {vdeh_config['estimated_records']:,}")

📄 VDEH Datenquelle: Neuerwerbungen der VDEH Bibliotheken
📂 Dateipfad: /media/sz/Data/Bibo/analysis/data/vdeh/raw/VDEH_mab_all.xml
🔤 Encoding: utf-8
📊 Geschätzte Records: 58,760


In [3]:
# 📊 XML-DATEN LADEN
vdeh_config = config.get('data_sources.vdeh')
xml_file_path = config.project_root / vdeh_config['path']
max_records = config.get('data_processing.vdeh_parser.max_records')

print(f"📂 Lade VDEH XML-Datei: {xml_file_path}")
print(f"📊 Max Records: {max_records or 'Alle (~58k)'}")

# Prüfe ob Datei existiert
if not xml_file_path.exists():
    raise FileNotFoundError(f"VDEH XML-Datei nicht gefunden: {xml_file_path}")

# VDEH Parser importieren
from parsers.vdeh_parser import parse_bibliography, analyze_bibliography_data, get_sample_records

# Daten laden
df_vdeh = parse_bibliography(str(xml_file_path), max_records=max_records)

print(f"\n✅ DataFrame erstellt: {len(df_vdeh):,} Records")
print(f"💾 Speicherverbrauch: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"📋 Spalten: {list(df_vdeh.columns)}")

2025-11-10 19:52:13 - parsers.vdeh_parser - INFO - Starting parser for bibliographic data from /media/sz/Data/Bibo/analysis/data/vdeh/raw/VDEH_mab_all.xml (118.6 MB)


📂 Lade VDEH XML-Datei: /media/sz/Data/Bibo/analysis/data/vdeh/raw/VDEH_mab_all.xml
📊 Max Records: Alle (~58k)


2025-11-10 19:52:15 - parsers.vdeh_parser - INFO - Found 58,760 OAI records


2025-11-10 19:52:16 - parsers.vdeh_parser - INFO - Processed 5,000 records


2025-11-10 19:52:16 - parsers.vdeh_parser - INFO - Processed 10,000 records


...

2025-11-10 19:52:19 - parsers.vdeh_parser - INFO - Processed 50,000 records


2025-11-10 19:52:19 - parsers.vdeh_parser - INFO - Processed 55,000 records


2025-11-10 19:52:19 - parsers.vdeh_parser - INFO - Successfully processed 58,760 records


2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - DataFrame created: 58,760 rows, 12 columns, 31.9 MB



✅ DataFrame erstellt: 58,760 Records
💾 Speicherverbrauch: 31.9 MB
📋 Spalten: ['id', 'title', 'authors', 'authors_affiliation', 'year', 'publisher', 'isbn', 'issn', 'authors_str', 'num_authors', 'authors_affiliation_str', 'num_authors_affiliation']


In [4]:
df_vdeh.iloc[0]           

id                                                   aleph-publish:000000004
title                      Untersuchung der Gleichgewichte zwischen flüss...
authors                                                      [Thielmann, R.]
authors_affiliation                                 [Europäische Kommission]
year                                                                  1983.0
publisher                                                         Luxembourg
isbn                                                                    None
issn                                                                    None
authors_str                                                    Thielmann, R.
num_authors                                                                1
authors_affiliation_str                               Europäische Kommission
num_authors_affiliation                                                    1
Name: 0, dtype: object

In [5]:
# Standardnummern-Analyse (ISBN/ISSN)
print("=== STANDARDNUMMERN ANALYSE ===")

# ISBN Analyse
try:
    isbn_total = df_vdeh['isbn'].notna().sum()
    print("\nISBN Analyse:")
    print(f"Anzahl Datensätze mit ISBN: {isbn_total:,}")
    print(f"Prozent der Datensätze mit ISBN: {(isbn_total/len(df_vdeh)*100):.2f}%")

    if isbn_total > 0:
        # ISBN Längenanalyse
        df_with_isbn = df_vdeh[df_vdeh['isbn'].notna()]
        if not df_with_isbn.empty:
            isbn_lengths = df_with_isbn['isbn'].astype(str).str.len().value_counts().sort_index()
            print("\nISBN Längen:")
            for length, count in isbn_lengths.items():
                print(f"   {length} Stellen: {count:,} ISBNs ({count/isbn_total*100:.1f}%)")

            print("\nBeispiel ISBNs:")
            try:
                sample_with_isbn = df_with_isbn.sample(n=min(3, len(df_with_isbn)), random_state=42)
                for _, row in sample_with_isbn.iterrows():
                    print(f"   ISBN: {row['isbn']} | Titel: {row['title'][:70] if row['title'] else 'N/A'}...")
            except ValueError as e:
                print("   Keine ISBN-Beispiele verfügbar.")
except Exception as e:
    print(f"Fehler bei der ISBN-Analyse: {str(e)}")

# ISSN Analyse
try:
    issn_total = df_vdeh['issn'].notna().sum()
    print("\nISSN Analyse:")
    print(f"Anzahl Datensätze mit ISSN: {issn_total:,}")
    print(f"Prozent der Datensätze mit ISSN: {(issn_total/len(df_vdeh)*100):.2f}%")

    if issn_total > 0:
        df_with_issn = df_vdeh[df_vdeh['issn'].notna()]
        if not df_with_issn.empty:
            print("\nBeispiel ISSNs:")
            try:
                sample_with_issn = df_with_issn.sample(n=min(3, len(df_with_issn)), random_state=42)
                for _, row in sample_with_issn.iterrows():
                    print(f"   ISSN: {row['issn']} | Titel: {row['title'][:70] if row['title'] else 'N/A'}...")
            except ValueError as e:
                print("   Keine ISSN-Beispiele verfügbar.")
except Exception as e:
    print(f"Fehler bei der ISSN-Analyse: {str(e)}")

# Verlagsanalyse
if 'publisher' in df_vdeh.columns:
    try:
        publisher_total = df_vdeh['publisher'].notna().sum()
        print("\n\n=== VERLAGSANALYSE ===")
        print(f"Anzahl Datensätze mit Verlag: {publisher_total:,}")
        print(f"Prozent der Datensätze mit Verlag: {(publisher_total/len(df_vdeh)*100):.2f}%")
        
        if publisher_total > 0:
            print("\nBeispiel Verlage:")
            df_with_publisher = df_vdeh[df_vdeh['publisher'].notna()]
            sample_publisher = df_with_publisher.sample(n=min(3, len(df_with_publisher)), random_state=42)
            for _, row in sample_publisher.iterrows():
                print(f"   Verlag: {row['publisher'][:60]}... | Titel: {row['title'][:50] if row['title'] else 'N/A'}...")
    except Exception as e:
        print(f"Fehler bei der Verlagsanalyse: {str(e)}")

=== STANDARDNUMMERN ANALYSE ===

ISBN Analyse:
Anzahl Datensätze mit ISBN: 11,415
Prozent der Datensätze mit ISBN: 19.43%

ISBN Längen:
   0 Stellen: 1 ISBNs (0.0%)
   1 Stellen: 1 ISBNs (0.0%)
   4 Stellen: 2 ISBNs (0.0%)
   7 Stellen: 2 ISBNs (0.0%)
   8 Stellen: 697 ISBNs (6.1%)
   9 Stellen: 22 ISBNs (0.2%)
   11 Stellen: 22 ISBNs (0.2%)
   12 Stellen: 13 ISBNs (0.1%)
   13 Stellen: 8,038 ISBNs (70.4%)
   14 Stellen: 1 ISBNs (0.0%)
   16 Stellen: 2 ISBNs (0.0%)
   17 Stellen: 2,485 ISBNs (21.8%)
   20 Stellen: 77 ISBNs (0.7%)
   21 Stellen: 1 ISBNs (0.0%)
   22 Stellen: 5 ISBNs (0.0%)
   23 Stellen: 31 ISBNs (0.3%)
   25 Stellen: 1 ISBNs (0.0%)
   26 Stellen: 9 ISBNs (0.1%)
   27 Stellen: 1 ISBNs (0.0%)
   28 Stellen: 1 ISBNs (0.0%)
   30 Stellen: 3 ISBNs (0.0%)

Beispiel ISBNs:
   ISBN: 3-826-56022-1 | Titel: Tribologische Untersuchungen zur Kaltmassivumformung...
   ISBN: 9-282-81437-8 | Titel: Mejora de las propiedades mecanicas de piezas forjadas y barras median...
   ISBN: 3-5

In [6]:
# 📈 BASIS-DATENANALYSE
print("📈 === VDEH DATENANALYSE ===\n")

# Basis-Analyse der VDEH-Daten
analyze_bibliography_data(df_vdeh)

2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - === BIBLIOGRAPHIC DATA ANALYSIS ===


2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - Total records: 58,760


2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - With title: 40,830 (69.5%)


2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - With authors: 17,011 (28.9%)


2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - With year: 33,687 (57.3%)


...

2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - Complete records: 16,013 (27.3%)


2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - Year range: 1900 - 2017


2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - Median year: 1998


2025-11-10 19:52:20 - parsers.vdeh_parser - INFO - Unique authors: 17,758


📈 === VDEH DATENANALYSE ===



In [7]:
# 💾 GELADENE DATEN EXPORTIEREN
processed_dir = config.project_root / config.get('paths.data.vdeh.processed')
processed_dir.mkdir(parents=True, exist_ok=True)

# Export als Parquet für nächste Pipeline-Stufe
output_path = processed_dir / '01_loaded_data.parquet'
df_vdeh.to_parquet(output_path, index=False)

print(f"💾 === DATEN LOADING ABGESCHLOSSEN ===")
print(f"✅ Geladene Daten exportiert: {output_path}")
print(f"📊 Records: {len(df_vdeh):,}")
print(f"📋 Spalten: {len(df_vdeh.columns)}")
print(f"💾 Dateigröße: {df_vdeh.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

# Metadaten für nächste Stufe
metadata = {
    'stage': '01_data_loading',
    'records_loaded': len(df_vdeh),
    'columns': list(df_vdeh.columns),
    'source_file': str(xml_file_path),
    'processing_date': pd.Timestamp.now().isoformat()
}

import json
metadata_path = processed_dir / '01_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"📄 Metadaten gespeichert: {metadata_path}")
print(f"\n➡️  Nächster Schritt: 02_vdeh_data_preprocessing.ipynb")

💾 === DATEN LOADING ABGESCHLOSSEN ===
✅ Geladene Daten exportiert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/01_loaded_data.parquet
📊 Records: 58,760
📋 Spalten: 12


💾 Dateigröße: 34.8 MB
📄 Metadaten gespeichert: /media/sz/Data/Bibo/analysis/data/vdeh/processed/01_metadata.json

➡️  Nächster Schritt: 02_vdeh_data_preprocessing.ipynb
